# Building Your Predictor

The next step after preparing and importing your data via `Getting_Data_Ready.ipynb` is to build your first model.

The overall process for this is:

* Setup
* Create a Predictor
* Deploy a Predictor
* Obtain a Forecast

To get started, simply execute the cells below:


## Setup


Import the standard Python Libraries that are used in this lesson.

In [1]:
import sys
import os
import time

import boto3

# importing forecast notebook utility from notebooks/common directory
#sys.path.insert( 0, os.path.abspath("../../common") )
import util

The line below will retrieve your shared variables from the first notebook.

In [3]:
%store -r

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [4]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

In [5]:
%store -r forecast_project_name
%store -r forecast_dataset_group_arn
%store -r forecast_dataset_arn
%store -r forecast_role_name
%store -r forecast_key
%store -r forecast_ds_import_job_arn

no stored variable or alias forecast_key
no stored variable or alias forecast_ds_import_job_arn


## Create a Predictor

Now in the previous notebook, your data was imported to be used by Forecast, here we will once again define your dataset information and then start building your model or predictor.

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [6]:
predictor_name = forecast_project_name + '_deeparp_algo'

In [7]:
forecast_horizon = 24

In [8]:
algorithm_arn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

In [9]:
create_predictor_response=forecast.create_predictor(PredictorName=predictor_name, 
                                                    AlgorithmArn=algorithm_arn,
                                                    ForecastHorizon=forecast_horizon,
                                                    PerformAutoML= False,
                                                    PerformHPO=False,
                                                    EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                           "BackTestWindowOffset": 24}, 
                                                    InputDataConfig= {"DatasetGroupArn": forecast_dataset_group_arn},
                                                    FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                          "Featurizations": 
                                                                          [
                                                                              {"AttributeName": "target_value", 
                                                                               "FeaturizationPipeline": 
                                                                                [
                                                                                  {"FeaturizationMethodName": "filling", 
                                                                                   "FeaturizationMethodParameters": 
                                                                                    {"frontfill": "none", 
                                                                                     "middlefill": "zero", 
                                                                                     "backfill": "zero"}
                                                                                  }
                                                                                ]
                                                                              }
                                                                          ]
                                                                         }
                                                 )

In [10]:
forecast_predictor_arn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=forecast_predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

CREATE_PENDING .
CREATE_IN_PROGRESS ....................................................................................................................................................................................................

### Get Error Metrics

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=forecast_predictor_arn)

## Create a Forecast

Now create a forecast using the model that was trained

In [ ]:
forecast_name = forecast_project_name + '_deeparp_algo_forecast'

In [ ]:
create_forecast_response=forecast.create_forecast(ForecastName=forecast_name,
                                                  PredictorArn=forecast_predictor_arn)
forecast_arn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

### Get Forecast

Once created, the forecast results are ready and you view them. 

In [ ]:
from pprint import pprint
print(forecast_arn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"client_12"}
)
pprint(forecastResponse)

## Next Steps

Now that your forecast has been created, to evaluate it use `3.Evaluating_Your_Predictor.ipynb` but before opening it, execute the cell below to share your variables again with the next notebook.

In [ ]:
%store forecast_arn
%store forecast_predictor_arn

# Building Your Predictor

The next step after preparing and importing your data via `Getting_Data_Ready.ipynb` is to build your first model.

The overall process for this is:

* Setup
* Create a Predictor
* Deploy a Predictor
* Obtain a Forecast

To get started, simply execute the cells below:


## Setup


Import the standard Python Libraries that are used in this lesson.

In [1]:
import sys
import os
import time

import boto3

# importing forecast notebook utility from notebooks/common directory
#sys.path.insert( 0, os.path.abspath("../../common") )
import util

The line below will retrieve your shared variables from the first notebook.

In [3]:
%store -r

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [4]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

In [5]:
%store -r forecast_project_name
%store -r forecast_dataset_group_arn
%store -r forecast_dataset_arn
%store -r forecast_role_name
%store -r forecast_key
%store -r forecast_ds_import_job_arn

no stored variable or alias forecast_key
no stored variable or alias forecast_ds_import_job_arn


## Create a Predictor

Now in the previous notebook, your data was imported to be used by Forecast, here we will once again define your dataset information and then start building your model or predictor.

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [6]:
predictor_name = forecast_project_name + '_deeparp_algo'

In [7]:
forecast_horizon = 24

In [8]:
algorithm_arn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

In [9]:
create_predictor_response=forecast.create_predictor(PredictorName=predictor_name, 
                                                    AlgorithmArn=algorithm_arn,
                                                    ForecastHorizon=forecast_horizon,
                                                    PerformAutoML= False,
                                                    PerformHPO=False,
                                                    EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                           "BackTestWindowOffset": 24}, 
                                                    InputDataConfig= {"DatasetGroupArn": forecast_dataset_group_arn},
                                                    FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                          "Featurizations": 
                                                                          [
                                                                              {"AttributeName": "target_value", 
                                                                               "FeaturizationPipeline": 
                                                                                [
                                                                                  {"FeaturizationMethodName": "filling", 
                                                                                   "FeaturizationMethodParameters": 
                                                                                    {"frontfill": "none", 
                                                                                     "middlefill": "zero", 
                                                                                     "backfill": "zero"}
                                                                                  }
                                                                                ]
                                                                              }
                                                                          ]
                                                                         }
                                                 )

In [10]:
forecast_predictor_arn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [12]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=forecast_predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

ACTIVE 


### Get Error Metrics

In [13]:
forecast.get_accuracy_metrics(PredictorArn=forecast_predictor_arn)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
   'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 18.117713949884383,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.2597559132546497},
       {'Quantile': 0.5, 'LossValue': 0.270472984787922},
       {'Quantile': 0.1, 'LossValue': 0.07720929228468607}],
      'ErrorMetrics': [{'ForecastType': 'mean',
        'WAPE': 0.2656715696251501,
        'RMSE': 18.117713949884383,
        'MASE': 2.2266954870808107,
        'MAPE': 0.29553923630341355}],
      'AverageWeightedQuantileLoss': 0.20247939677575258}},
    {'TestWindowStart': datetime.datetime(2014, 10, 30, 0, 0, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2014, 10, 31, 0, 0, tzinfo=tzlocal()),
     'ItemCount': 3,
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 18.117713949884383,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.2597559132

## Create a Forecast

Now create a forecast using the model that was trained

In [14]:
forecast_name = forecast_project_name + '_deeparp_algo_forecast'

In [15]:
create_forecast_response=forecast.create_forecast(ForecastName=forecast_name,
                                                  PredictorArn=forecast_predictor_arn)
forecast_arn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [16]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

CREATE_PENDING 
CREATE_IN_PROGRESS ...............................................................................................................................................................................................
ACTIVE 


### Get Forecast

Once created, the forecast results are ready and you view them. 

In [17]:
from pprint import pprint
print(forecast_arn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"client_12"}
)
pprint(forecastResponse)

arn:aws:forecast:us-east-1:992382405090:forecast/util_power_forecast_deeparp_algo_forecast

{'Forecast': {'Predictions': {'p10': [{'Timestamp': '2014-10-31T00:00:00',
                                       'Value': 46.6488418579},
                                      {'Timestamp': '2014-10-31T01:00:00',
                                       'Value': 44.0505981445},
                                      {'Timestamp': '2014-10-31T02:00:00',
                                       'Value': 43.3063316345},
                                      {'Timestamp': '2014-10-31T03:00:00',
                                       'Value': 34.5210533142},
                                      {'Timestamp': '2014-10-31T04:00:00',
                                       'Value': 32.3293800354},
                                      {'Timestamp': '2014-10-31T05:00:00',
                                       'Value': 31.18542099},
                                      {'Timestamp': '2014-10-31T06:00:00',
 

## Next Steps

Now that your forecast has been created, to evaluate it use `3.Evaluating_Your_Predictor.ipynb` but before opening it, execute the cell below to share your variables again with the next notebook.

In [18]:
%store forecast_arn
%store forecast_predictor_arn

Stored 'forecast_arn' (str)
Stored 'forecast_predictor_arn' (str)
